# Sentiment Analysis using Deep Learning

This is a short example of how to use the Aqueduct Platform to deploy a sentiment analysis model.

In [1]:
import aqueduct 
from aqueduct.decorator import op, check

In [2]:
api_key = "EGUJVMZIPL05RA6T7BFHQWDKCO1S43N2"

# You can use localhost if you're running this notebook on the same machine as your server.
address = "localhost:8080"

In [3]:
client = aqueduct.Client(api_key, address)

## Getting the Input Data

In [4]:
warehouse = client.integration("aqueduct_demo")
reviews_table = warehouse.sql("select * from hotel_reviews;")

In [5]:
reviews_table.get()

,hotel_name,review_date,reviewer_nationality,review
0,H10 Itaca,2017-08-03,Australia,Damaged bathroom shower screen sealant and ti...
1,De Vere Devonport House,2016-03-28,United Kingdom,No Negative The location and the hotel was ver...
2,Ramada Plaza Milano,2016-05-15,Kosovo,No Negative Im a frequent traveler i visited m...
3,Aloft London Excel,2016-11-05,Canada,Only tepid water for morning shower They said ...
4,The Student Hotel Amsterdam City,2016-07-31,Australia,No Negative The hotel had free gym table tenni...
...,...,...,...,...
95,The Chesterfield Mayfair,2015-08-25,Denmark,Bad Reading light And light in bathNo Positive
96,Hotel V Nesplein,2015-08-27,Turkey,Nothing except the construction going on the s...
97,Le Parisis Paris Tour Eiffel,2015-10-20,Australia,When we arrived we had to bring our own baggag...
98,NH Amsterdam Museum Quarter,2016-01-26,Belgium,No stairs even to go the first floor Restaura...


## Applying the Model

Here we use the HuggingFace Transformers package to apply a standard sentiment analysis model to our data.

In [6]:
from transformers import pipeline
import pandas as pd
import torch # this is needed to ensure that pytorch is installed.

@op()
def sentiment_prediction(reviews):
    model = pipeline("sentiment-analysis")
    return reviews.join(pd.DataFrame(model(list(reviews['review']))))

/Users/kennethxu/opt/anaconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
sentiment_table = sentiment_prediction(reviews_table)

Getting a preview of the results.

In [8]:
sentiment_table.get()

,hotel_name,review_date,reviewer_nationality,review,label,score
0,H10 Itaca,2017-08-03,Australia,Damaged bathroom shower screen sealant and ti...,POSITIVE,0.715812
1,De Vere Devonport House,2016-03-28,United Kingdom,No Negative The location and the hotel was ver...,POSITIVE,0.999741
2,Ramada Plaza Milano,2016-05-15,Kosovo,No Negative Im a frequent traveler i visited m...,POSITIVE,0.999773
3,Aloft London Excel,2016-11-05,Canada,Only tepid water for morning shower They said ...,NEGATIVE,0.999169
4,The Student Hotel Amsterdam City,2016-07-31,Australia,No Negative The hotel had free gym table tenni...,NEGATIVE,0.931378
...,...,...,...,...,...,...
95,The Chesterfield Mayfair,2015-08-25,Denmark,Bad Reading light And light in bathNo Positive,NEGATIVE,0.999340
96,Hotel V Nesplein,2015-08-27,Turkey,Nothing except the construction going on the s...,POSITIVE,0.999691
97,Le Parisis Paris Tour Eiffel,2015-10-20,Australia,When we arrived we had to bring our own baggag...,NEGATIVE,0.999032
98,NH Amsterdam Museum Quarter,2016-01-26,Belgium,No stairs even to go the first floor Restaura...,POSITIVE,0.996806


## Publishing the Workflow
Saving the predictions back to the data warehouse.

In [9]:
sentiment_table.save(warehouse.config(table='sentiment_pred', update_mode='replace'))

Publishing the flow to run every hour.

In [10]:
sentiment_flow = client.publish_flow(name = "Demo Customer Sentiment", 
                                   artifacts = [sentiment_table],
                                   schedule = aqueduct.hourly())

KeyError: '13f6136a-9beb-4084-9061-5d53a87645e7'

In [ ]:
this is clearly a bug